# Coronavirus publications 

* Dimensions publications matching the keyword 'coronavirus' are counted and segmented by whether the keyword appears in the title, abstract, or full text
* Yearly distribution is calculated too

In [22]:
# @markdown # Get the API library and login 
# @markdown Click the 'play' button on the left (or shift+enter) after entering your API credentials and search term below
# @markdown ---

username = "" #@param {type: "string"}
password = "" #@param {type: "string"}
endpoint = "https://sn-insights.dimensions.ai" #@param {type: "string"}

!pip install dimcli plotly_express -U --quiet 
import dimcli
from dimcli.shortcuts import *
dimcli.login(username, password, endpoint)
dsl = dimcli.Dsl()

#
# load common libraries
import pandas as pd
import plotly.express as px

# @markdown ---
# @markdown # Enter a search term 
KEYWORD = "coronavirus" #@param {type: "string"}


DimCli v0.6.2.3 - Succesfully connected to <https://app.dimensions.ai> (method: manual login)


In [65]:

q = """search publications in {} for "{}" return publications limit 1"""

results = { 
    "title_only" : 0 ,
    "title_abstract_only" : 0 ,
    "full_data" : 0, 
    "full_data_minus_title_or_abstract" : 0
    }

for x in results:
  if x != "full_data_minus_title_or_abstract":
    t = dsl.query(q.format(x, KEYWORD), verbose=False)
    results[x] = t.count_total

# build dataframe
results["full_data_minus_title_or_abstract"] = results["full_data"] - results["title_abstract_only"]
d = {'location': list(results.keys()), 'count': list(results.values())}
df1 = pd.DataFrame.from_dict(d)

# display
px.bar(df1, x="location", y="count", title=f"Publications matching the topic '{KEYWORD}'")

## Yearly Distribution

In [78]:
q2 = """search publications in {} for "{}" return year limit 100"""

results = { 
    "title_only" : 0 ,
    "title_abstract_only" : 0 ,
    "full_data" : 0, 
    }

df_years = pd.DataFrame(columns=['id', 'count', 'location'])
for x in results:
  dftemp = dsl.query(q2.format(x, KEYWORD), verbose=False).as_dataframe()
  dftemp['location'] = x
  df_years = df_years.append(dftemp, ignore_index=True)

# clean up
df_years = df_years.rename(columns={"id": "year"})
df_years = df_years.sort_values(["location", "year"]) 

# display
px.line(df_years, x="year", y="count", 
        color="location", 
        title=f"Yearly distribution of publications matching the topic '{KEYWORD}'")

## Save and Download the data

In [0]:
# save to csv
df1.to_csv("results_total.csv")
df_years.to_csv("results_years.csv")



Download the data - PS this last step is for Google Colab users only.

In [0]:
from google.colab import files

try:
  # try to download from colab: sometimes it fails hence print a message
  time.sleep(5)
  files.download("results_total.csv")
  files.download("results_years.csv") 
except:
  print("Google Colab failed to download - please try again.")
